In [1]:
from models import models

# Load Case and Data

In [2]:
CaseName = "simple.cas.h5"
DataName = None #"simple.dat.h5"

# Activate Models

with ON/OFF model just say write "yes" or "no". With models that needs to have a specific parameter set just write that as in the TUI

In [3]:
# Models
# Set None if not wanted

energy = "yes" # "no"
viscous = "ke-realizable" #"Laminar" /  "ke-realizable"
radiation = "DO" 

# Operating Conditions

In [4]:
# Operating Conditions

operatingPressure = 1e5
gravity = None
x_gravity = 0
y_gravity = -9.81
z_gravity = 0

# Solution Information

In [5]:
# Solution Data

numberSolutionSegment = 2
numberIteration = [100,2000] # size equal to value of numberSolutionSegment

# Schemes 
schemes = ["first order", "second order"]
radiationIteration = [50 , 10]

# Convergence Criteria

In [6]:
# Convergence Criteria
continuity = 1e-5
xVel = 1e-5
yVel = 1e-5
zVel = 1e-5
energyRes = 1e-10
kappa = 1e-5
epsilon = 1e-5
radiationRes = 1e-10

In [7]:
with open('journal.jou', 'w') as file:
    # CheckPoint Name
    file.write("(set! checkpoint/check-filename './AC3D.txt')\n")
    
    # Read Case
    file.write(f"; ------------------- READ CASE -------------------\nread-case {CaseName}\n")
    
    # Read Data
    if DataName != None:
        file.write(f"; ------------------- READ DATA -------------------\nread-data {DataName}\n")
        
    # Models
    file.write(f"; ------------------- ACTIVATE MODELS -------------------\n")
    file.write(models["Viscous"][viscous])
    if energy == "yes":
        file.write(models["Energy"])
    if radiation != None:
        file.write(models["Radiation"][radiation])
    
    # Convergence Criteria
    file.write(f"; ------------------- RESIDUALS -------------------\n")
    file.write("/solve/monitors/residual convergence-criteria\n")
    file.write(f"{continuity}\n{xVel}\n{yVel}\n{zVel}\n")
    if viscous != "Laminar":
        file.write(f"{kappa}\n{epsilon}\n")
    if energy != None:
        file.write(f"{energyRes}\n")
    if radiation != None:
        file.write(f"{radiationRes}\n")
        
    # Operating Conditions
    file.write(f"; ------------------- Operating Conditions -------------------\n")
    file.write(f"/define/operating-conditions operating-pressure\n{operatingPressure}\n")
    if gravity == "yes":
        file.write(f"/define/operating-conditions gravity\nyes\n")
        file.write(f"{x_gravity}\n{y_gravity}\n{z_gravity}\n")
        
    # Schemes
    # Initializa as SIMPLE without pseudo-timestep 
    file.write(f";------------------- Schemes -------------------\n/solve/set/discretization-scheme pressure\n14\n")
    file.write(f"/solve/set gradient-schem\nno\nno\n")
    file.write(f"/solve/set p-v-coupling\n20\n")
    file.write(f"/solve/set/pseudo-time-method formulation\n0\n")
    
    file.write(";------------------- INITIALIZE -------------------\n")
    file.write("/solve/initialize initialize-flow\n")
    file.write(";------------------- SOLUTION -------------------\n")
    
    for i in range(numberSolutionSegment):
        file.write(f";------------------- {i+1} ITERATION ROUND -------------------\n")
        if schemes[i] == "first order":
            order = 0
        else:
            order = 1
        # momentum
        file.write(f"/solve/set/discretization-scheme mom\n{order}\n")
        # check for turbulence
        if viscous == "ke-realizable":
            file.write(f"solve/set/discretization-scheme k\n{order}\n/solve/set/discretization-scheme epsilon\n{order}\n")
        if energy != None:
            file.write(f"solve/set/discretization-scheme temperature\n{order}\n")
        if radiation != None:
            file.write(f"solve/set/discretization-scheme disco\n{order}\n")
            file.write(f"/define/models/radiation radiation-model-parameters\n{radiationIteration[i]}\n")
            
        file.write(f"it {numberIteration[i]}\n")
        file.write("yes\n")


    file.write(";------------------- SAVING -------------------\n")
    file.write("write-data converged.dat.gz\n")
    file.write("write-case converged.cas.gz\n")
    
    file.write(";------------------- CLOSING FLUENT -------------------\n")
    file.write("exit\nyes")
